In [12]:
import os 
from skyfield.api import wgs84, load
def getTLE(satName):
    # Define a list of URLs to check for TLE data
    urls = [
        'http://celestrak.org/NORAD/elements/weather.txt',
        'http://celestrak.org/NORAD/elements/amateur.txt',
        'http://celestrak.org/NORAD/elements/cubesat.txt'
    ]

    # Attempt to load TLE data from each URL
    for url in urls:
        try:
            os.chdir('G:\MinorProject\IoTRACK\\built23JUN\op')
            satellites = load.tle_file(url)
            by_name = {sat.name: sat for sat in satellites}
            if satName in by_name:
                return by_name[satName]
        except Exception as e:
            print(f"Error while fetching TLE from {url}: {e}")

    print(f"Satellite '{satName}' not found in any of the provided URLs.")
    return None
satName = input("Enter the satellite name: ")
getTLE(satName)


<EarthSatellite ISS (ZARYA) catalog #25544 epoch 2023-10-23 04:26:45 UTC>

In [14]:
from geopy import Nominatim
import sqlite3
from sqlite3 import Error
import tqdm, os
from skyfield.api import wgs84, load
import numpy as np
import pandas as pd
import datetime, pytz
from tabulate import tabulate
from utils import getTLE, getLocation, create_connection, predict, predictPrecise

SATNAME = 'NOAA 18'
LOCATION = 'Chennai'

ts = load.timescale()
t = ts.now()

def getPredictedPath(GROUP_ID):
    conn = create_connection()
    if conn:
        try:
            cursor = conn.cursor()

            cursor.execute(f"SELECT timeUTC from predicted_path WHERE group_id ={GROUP_ID} AND name = 'AOS' ")   
            ROW = cursor.fetchone()

            if ROW : 
                predictionStartTime = ROW[0].strip('`')

            cursor.execute(f"SELECT timeUTC from predicted_path WHERE group_id ={GROUP_ID} AND name = 'LOS' ")  
            ROW = cursor.fetchone()

            if ROW : 
                predictionEndTime = ROW[0].strip('`')
            
            cursor.execute(f"SELECT date from predicted_path WHERE group_id ={GROUP_ID} AND name = 'AOS' ")   
            ROW = cursor.fetchone()

            if ROW : 
                _date = ROW[0].strip('`')

            conn.close()
            return predictionStartTime, predictionEndTime , _date
        except Exception as e:
            print(f"An error occurred while getting the predicted path: {e}")
            conn.close()
            return None
    return None
def predictPrecise():
    try:
        whichGroup = int(input("Enter the group number: "))

        ps , pe , date = getPredictedPath(whichGroup)

        predictionStartTime = datetime.datetime.strptime(ps, '%H:%M:%S')
        predictionEndTime = datetime.datetime.strptime(pe, '%H:%M:%S')
        ps_date_formatted = datetime.datetime.strptime(date, '%d-%b-%y')

        ps_formatted = predictionStartTime.strftime('%H%M%S')
        pe_formatted = predictionEndTime.strftime('%H%M%S')         
        ps_date_formatted = ps_date_formatted.strftime('%Y%m%d')

        pe_start_hour = int(pe_formatted[0:2])
        pe_start_min = int(pe_formatted[2:4])
        pe_start_sec = int(pe_formatted[4:6])

        ps_end_hour = int(ps_formatted[0:2])
        ps_end_min = int(ps_formatted[2:4])
        ps_end_sec = int(ps_formatted[4:6])

        ps_date_year = int(ps_date_formatted[0:4])
        ps_date_month = int(ps_date_formatted[4:6])
        ps_date_day = int(ps_date_formatted[6:8])
        
        print(ps_date_year, ps_date_month, ps_date_day)
        print( ps_end_hour, ps_end_min, ps_end_sec)
        print(pe_start_hour, pe_start_min, pe_start_sec)
        ist_timezone = pytz.timezone('Asia/Kolkata')

        satellite = getTLE(SATNAME)
        latNS, logEW = getLocation(LOCATION) 
        bpl = wgs84.latlon(latNS, logEW )

        difference = satellite - bpl

        predictionEndTime = ts.utc(ps_date_year, ps_date_month, ps_date_day, pe_start_hour, pe_start_min, pe_start_sec)
        predictionStartTime = ts.utc(ps_date_year, ps_date_month,  ps_date_day, ps_end_hour, ps_end_min, ps_end_sec)
            
        interval = 2   
        
        current_time = predictionStartTime
        predictedPath = []
        conn = create_connection()
        while current_time.tt <= predictionEndTime.tt:
            ti = current_time
            difference = satellite - bpl
            topocentric = difference.at(ti)
            alt, az, distance = topocentric.altaz()


            if  alt.degrees >= 0 :
                
                ti_ist = ti.astimezone(ist_timezone)
                ist_date = ti_ist.strftime('%d-%b-%y')
                ist_time = ti_ist.strftime('%H:%M:%S')

                path = {
                    'date': ist_date,
                    'timeIST': ist_time,
                    'timeUTC': ti.utc_strftime('%H:%M:%S'),
                    'azi': np.round(az.degrees, 2),
                    'elev': np.round(alt.degrees, 2)
                }

                predictedPath.append(path)

            # Increment the current time by the interval
            current_time += datetime.timedelta(seconds=interval)
    
            
        if conn:
            cursor = conn.cursor()
            cursor.execute("DROP TABLE IF EXISTS precisePredict")  # Clear the existing table if it exists
            cursor.execute(
                """
                CREATE TABLE IF NOT EXISTS precisePredict (
                
                    date TEXT,
                    timeIST TEXT,
                    timeUTC TEXT,
                    azi REAL,
                    elev REAL
                )
                """
            )  # Create the table to store the predicted path

            for path in predictedPath:
                cursor.execute(
                    """
                    INSERT INTO precisePredict ( date, timeIST ,timeUTC, azi, elev)
                    VALUES ( ?, ?,?, ?, ?)
                    """,
                    (   
                        
                        path['date'],
                        path['timeIST'],
                        path['timeUTC'],
                        path['azi'],
                        path['elev']
                    ),
                )  # Insert each path into the table

            conn.commit()
            conn.close()

            # print(ps_date_formatted)         
            # print(type(ps_date_formatted))
            # print( pe_start_hour, pe_start_min, pe_start_sec)
            # print( ps_end_hour, ps_end_min, ps_end_sec)
            # print( ps_formatted, pe_formatted)
            # print( type(ps_formatted), type(pe_formatted))

            print("done")
 
    except Exception as e:
        print(f"An error occurred while predicting the path: {e}")

predictPrecise()

2023 10 23
4 36 3
4 51 5
done
